In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data= pd.read_csv("../input/tabular-playground-series-may-2021/train.csv")

In [ ]:
print(data.shape[0])
print(data.shape[1])

print(data.columns)


In [ ]:
data.head()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import time
import warnings
import numpy as np
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
from sklearn.manifold import TSNE
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics._classification import accuracy_score, log_loss
from sklearn.linear_model import SGDClassifier
from imblearn.over_sampling import SMOTE
from collections import Counter
from scipy.sparse import hstack
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold 
from collections import Counter, defaultdict
from sklearn.calibration import CalibratedClassifierCV
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
import math
from sklearn.metrics import normalized_mutual_info_score
from sklearn.ensemble import RandomForestClassifier
warnings.filterwarnings("ignore")

from mlxtend.classifier import StackingClassifier

from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

In [ ]:
X = pd.DataFrame(data.drop("target", axis = 1))

In [ ]:
X = X[['feature_38','feature_14','feature_31','feature_15','feature_34','feature_28','feature_9','feature_19','feature_24','feature_48','feature_7','feature_35','feature_17','feature_6','feature_43','feature_40','feature_12','feature_42','feature_3','feature_18','feature_37','feature_41','feature_49','feature_33']]

In [ ]:
#X['sum_feature'] = X['feature_39'] + X['feature_23']+X['feature_25']+X['feature_46']+X['feature_16']


In [ ]:
X.describe()

In [ ]:
#data['Employed'] = data['Employed'].replace('Fulltime',1).replace('Parttime',0)
y

In [ ]:
#data['Employed'] = data['Employed'].replace('Fulltime',1).replace('Parttime',0)

lencoder = LabelEncoder()
y = pd.DataFrame(lencoder.fit_transform(data['target']), columns=['target'])


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, stratify=Y, test_size=0.2)

x_train, x_cv, y_train, y_cv = train_test_split(X_train, Y_train, stratify=Y_train, test_size=0.2)

In [ ]:
print(x_train.shape[0],x_cv.shape[0],X_test.shape[0])
x_train.head()
y_train.head()

In [ ]:



train_class_distribution = pd.DataFrame(y_train).value_counts()
test_class_distribution = pd.DataFrame(Y_test).value_counts()
cv_class_distribution = pd.DataFrame(y_cv).value_counts()
my_colors = 'rgbkymc'
train_class_distribution.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Data points per Class')
plt.title('Distribution of yi in train data')
plt.grid()
plt.show()

my_colors = 'rgbkymc'
test_class_distribution.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Data points per Class')
plt.title('Distribution of yi in train data')
plt.grid()
plt.show()

my_colors = 'rgbkymc'
cv_class_distribution.plot(kind='bar')
plt.xlabel('Class')
plt.ylabel('Data points per Class')
plt.title('Distribution of yi in train data')
plt.grid()
plt.show()


In [ ]:
from xgboost import XGBClassifier
from xgboost import plot_importance
from matplotlib import pyplot

pyplot.figure(figsize=(50,30))
model = XGBClassifier()
model.fit(x_train, y_train)
plot_importance(model, max_num_features=25)

pyplot.show()

In [ ]:
alpha = [10 ** x for x in range(-5, 1)] # hyperparam for SGD classifier.



cv_log_error_array=[]
for i in alpha:
    clf = SGDClassifier(alpha=i, penalty='l2', loss='log', random_state=42, class_weight='balanced')
    clf.fit(x_train, y_train)
    sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
    sig_clf.fit(x_train, y_train)
    predict_y = sig_clf.predict_proba(x_cv)
    cv_log_error_array.append(log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
    print('For values of alpha = ', i, "The log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))

fig, ax = plt.subplots()
ax.plot(alpha, cv_log_error_array,c='g')
for i, txt in enumerate(np.round(cv_log_error_array,3)):
    ax.annotate((alpha[i],np.round(txt,3)), (alpha[i],cv_log_error_array[i]))
plt.grid()
plt.title("Cross Validation Error for each alpha")
plt.xlabel("Alpha i's")
plt.ylabel("Error measure")
plt.show()


best_alpha = np.argmin(cv_log_error_array)
clf = SGDClassifier(alpha=alpha[best_alpha], penalty='l2', loss='log', random_state=42, class_weight="balanced")
clf.fit(x_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(x_train, y_train)

predict_y = sig_clf.predict_proba(x_train)
print('For values of best alpha = ', alpha[best_alpha], "The train log loss is:",log_loss(y_train, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(x_cv)
print('For values of best alpha = ', alpha[best_alpha], "The cross validation log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_test)
print('For values of best alpha = ', alpha[best_alpha], "The test log loss is:",log_loss(Y_test, predict_y, labels=clf.classes_, eps=1e-15))


In [ ]:
clf = SGDClassifier(alpha=0.1, penalty='l2', loss='log', random_state=42, )
clf.fit(x_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(x_train, y_train)
#predict_y = sig_clf.predict_proba(X_test)

In [ ]:
result = sig_clf.predict(test)
outputArray = []
id = 100000
for i in range(len(result)):
    predict = result[i]
    innerArray = [id, predict[0], predict[1], predict[2], predict[3]]
    outputArray.append(innerArray)
    id += 1
df = pd.DataFrame(outputArray, columns=['id', 'Class_1', 'Class_2', 'Class_3', 'Class_4'])
df.to_csv(path_or_buf='submission.csv', index=False)
df

In [ ]:

sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv", index_col="id")
sub[["Class_1","Class_2","Class_3","Class_4"]] = sig_clf.predict_proba(test)
sub.to_csv("lr_onehot.csv")

In [ ]:


alpha = [10,50,100,200,500,1000]
max_depth = [2,3,5,10]
cv_log_error_array = []
for i in alpha:
    for j in max_depth:
        print("for n_estimators =", i,"and max depth = ", j)
        clf = LGBMClassifier(n_estimators=i, criterion='gini', max_depth=j, random_state=42, n_jobs=-1)
        clf.fit(x_train, y_train)
        sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
        sig_clf.fit(x_train, y_train)
        sig_clf_probs = sig_clf.predict_proba(x_cv)
        cv_log_error_array.append(log_loss(y_cv, sig_clf_probs, labels=clf.classes_, eps=1e-15))
        print("Log Loss :",log_loss(y_cv, sig_clf_probs)) 

best_alpha = np.argmin(cv_log_error_array)
clf = LGBMClassifier(n_estimators=alpha[int(best_alpha/4)], , max_depth=max_depth[int(best_alpha%4)], random_state=42, n_jobs=-1)
clf.fit(x_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(x_train, y_train)

predict_y = sig_clf.predict_proba(x_train)
print('For values of best alpha = ', alpha[int(best_alpha/4)], "The train log loss is:",log_loss(y_train, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(x_cv)
print('For values of best alpha = ', alpha[int(best_alpha/4)], "The cross validation log loss is:",log_loss(y_cv, predict_y, labels=clf.classes_, eps=1e-15))
predict_y = sig_clf.predict_proba(X_test)
print('For values of best alpha = ', alpha[int(best_alpha/4)], "The test log loss is:",log_loss(Y_test, predict_y, labels=clf.classes_, eps=1e-15))

In [ ]:

clf = LGBMClassifier(n_estimators=alpha[int(best_alpha/4)], max_depth=max_depth[int(best_alpha%4)], random_state=42, n_jobs=-1)
clf.fit(x_train, y_train)
sig_clf = CalibratedClassifierCV(clf, method="sigmoid")
sig_clf.fit(x_train, y_train)

In [ ]:

sub = pd.read_csv("/kaggle/input/tabular-playground-series-may-2021/sample_submission.csv", index_col="id")
sub[["Class_1","Class_2","Class_3","Class_4"]] = sig_clf.predict_proba(test)
sub.to_csv("light1234.csv")

In [ ]:
test=pd.read_csv("../input/tabular-playground-series-may-2021/test.csv")

In [ ]:
test.describe()

In [ ]:
test = test[['feature_38','feature_14','feature_31','feature_15','feature_34','feature_28','feature_9','feature_19','feature_24','feature_48','feature_7','feature_35','feature_17','feature_6','feature_43','feature_40','feature_12','feature_42','feature_3','feature_18','feature_37','feature_41','feature_49','feature_33']]

In [ ]:
#test['sum_feature'] = test['feature_39'] + test['feature_23']+test['feature_25']+test['feature_46']+test['feature_16']


In [ ]:
test.describe()

In [ ]:
!pip install lightgbm


In [ ]:
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier